## 빅데이터 실습

### 제주도 핫플레이스 시각화

#### 데이터 불러오기

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import unicodedata  # 유니코드 데이터 처리 모듈
import re   # 정규식 처리 모듈
import time
from PIL import Image,ImageOps
from tqdm import tqdm
from collections import Counter
from wordcloud import WordCloud
import platform
import folium
from folium.plugins import MarkerCluster # 마커클러스터용 클래스 추가
import requests # 인터넷에서 받아오는 라이브러리

In [ ]:
# 한글 깨짐 문제 해결
from matplotlib import rcParams, font_manager, rc

font_path = 'C:/Windows/Fonts/malgun.ttf'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font',family=font)
rcParams['axes.unicode_minus'] = False

In [ ]:
#불필요한 경고메시지 제거
import warnings
warnings.filterwarnings('ignore')   

In [ ]:
## 데이터 불러오기
rawTotal = pd.read_excel('../day04/data/1_crawling_raw.xlsx')
rawTotal

In [ ]:
# 위치정보 확인
locCounts = rawTotal['place'].value_counts()
locCounts.head(5)

In [ ]:
# DF로 변경
dfLocCounts = pd.DataFrame(locCounts)
dfLocCounts.head(5)

In [ ]:
dfLocCounts.to_excel('./data/3_location_counts.xlsx')

In [ ]:
Location = list(dfLocCounts.index)
Location

##### 카카오 OPEN API로 장소 검색
- 카카오 개발자 사이트에서 애플리케이션 등록
- key를 받음

In [ ]:
shopName = '고래배꼽'
## keyword는 가게명,  address는 가게 주소로 검색
url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={shopName}'

#header = {'Authorization' : 'KakaoAK 056c8acfa7a0c735d59a6d2d9b97502c'}

places = requests.get(url, headers=header).json()['documents'][0]
places

In [ ]:
## 장소 검색 함수 findPlace
def findPlace(shopName):
    # 1. 검색API 주소변수
    url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={shopName}'
    # 2. header 구성
    header
    # 3. API를 웹으로 요청, 결과 리턴
    places = requests.get(url, headers=header).json()['documents'][0]
    # 4. 필요정보만 추출
    realName = places['place_name']
    lat = places['y'] # 위도(latitude)
    lng = places['x'] # 경도(longgitude)
    phoneNumber = places['phone']
    realAddress = places['address_name']

    return[shopName, realName,phoneNumber,realAddress,lat,lng]

In [ ]:
findPlace('제주 공항')

In [ ]:
# Locations 리스트를 반복문으로 주소가져오는 처리
locationResults = []

for loc in tqdm(Location):
    try:
        data = findPlace(loc)
        locationResults.append(data)
        time.sleep(0.3)
    except:
        pass

In [ ]:
## list --> DataFrame 변경
dfLocationResult = pd.DataFrame(locationResults)

In [ ]:
dfLocationResult.columns=['초기이름','실제이름','전화번호','주소','위도','경도']

In [ ]:
dfLocationResult.to_excel('./data/3_locations.xlsx',index=False)

#### 인스타그램 크롤링정보와 카카오API로 주소 검색결과정보 병함

In [ ]:
dfLocCounts.reset_index(inplace=True)

In [ ]:
dfLocationResult.head()

In [ ]:
dfLocCounts.head()

In [ ]:
locationData = pd.merge(left=dfLocationResult,right=dfLocCounts, how='inner',left_on='초기이름',right_on='place')

In [ ]:
locationData.tail()

In [ ]:
# 중복데이터 확인
locationData['실제이름'].value_counts()

In [ ]:
# 중복데이터 pivot
locationDataFinal = locationData.pivot_table(values='count',index=['실제이름','주소','위도','경도'],aggfunc='sum')

In [ ]:
locationDataFinal.reset_index(inplace=True)

In [ ]:
locationDataFinal.to_excel('./data/3_locations_final.xlsx')

In [ ]:
## 위도, 경도가 object에서 float64로 변경
locationDataFinal = locationDataFinal.astype({'위도':'float64','경도':'float64'})

##### 지도 시각화
- folium 사용    

In [ ]:
locHanlaMt = [33.362050,126.530087]
map = folium.Map(location=locHanlaMt,zoom_start=11,tiles='OpenStreetMap')

for i in range(len(locationDataFinal)): #709번 반족
    name = locationDataFinal['실제이름'][i] # DF 컬럼이름 행값 == i
    count = int(locationDataFinal['count'][i])
    lat = float(locationDataFinal['위도'][i])
    lng = float(locationDataFinal['경도'][i])
    folium.CircleMarker(location=[lat,lng],radius=count,color='red',popup=name).add_to(map)
map

In [ ]:
map.save('./data/3_jeju_hotplaces.html')

In [ ]:
## 마커 클러스터
mapLocation = []
mapNames = []
for i in range(len(locationDataFinal)):
    data = locationDataFinal.iloc[i]    # 행 하나씩 읽어옴
    mapLocation.append((float(data['위도']),float(data['경도'])))
    mapNames.append(data['실제이름'])

locHanlaMt = [33.362050,126.530087]
map = folium.Map(location=locHanlaMt, zoom_start=11, tiles='OpenStreetMap')

markerCluster = MarkerCluster(locations=mapLocation, popups=mapNames, name='제주 핫플레이스', overlay=True, control=True)
markerCluster.add_to(map)
folium.LayerControl().add_to(map)

map.save('./data/3_jeju_hotplaces_cluster.html')